In [2]:
import csv
import pandas as pd
import os
import re

import math
from tqdm import tqdm
import multiprocessing as mp
import time
import sched
import ast
import collections

import numpy as np
from itertools import groupby


folder_prefix = '/home/jovyan/work/'

replace unicode spaces with actual 

bunch of junk words in text in WEBTEXT, 

using word_embedding script and the expanded stop word list on yoon's notebook

create a funciton that takes in a string
	- replace unicode spaces with actual space
	- numbers, 
	- "/" "-"
	
-data exploration, bar charts, word cloud

take in a string and return a 


Look at ping on basecamp
re.split('\W+|_', p)

In [5]:
cleaned_df = pd.read_pickle(folder_prefix + 'nowdata/charters_2015.pkl')